In [76]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

In [173]:
mut = pd.read_csv('TCGA.HNSC.mutations.txt', sep='\t', header=0, index_col = [0])
mut

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,...,SIFT,PolyPhen,DOMAINS,IMPACT,VARIANT_CLASS,FILTER,COSMIC,CALLERS,CONTEXT,NCALLERS
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,PIP4K2A,0,.,GRCh37,10,22830863,22830863,+,Silent,SNP,...,.,.,"Low_complexity_(Seg):seg,PROSITE_profiles:PS51...",LOW,SNV,common_in_exac,NONE,MUTECT|VARSCANS,CTCTCGCCCTC,2
TCGA-4P-AA8J,NRP1,0,.,GRCh37,10,33495214,33495214,+,Intron,SNP,...,.,.,.,MODIFIER,SNV,PASS,NONE,MUTECT|RADIA|SOMATICSNIPER|MUSE|VARSCANS,CATCTCTTCTT,5
TCGA-4P-AA8J,PFKFB3,0,.,GRCh37,10,6257188,6257188,+,Silent,SNP,...,.,.,"Superfamily_domains:SSF52540,PIRSF_domain:PIRS...",LOW,SNV,PASS,NONE,RADIA|MUTECT|MUSE|VARSCANS,GTGTTCAACGT,4
TCGA-4P-AA8J,VCL,0,.,GRCh37,10,75849918,75849918,+,Silent,SNP,...,.,.,"hmmpanther:PTHR18914:SF22,hmmpanther:PTHR18914...",LOW,SNV,PASS,BUFFER|p.A440D|c.1319C>A|3,RADIA|MUTECT|MUSE|VARSCANS,GAAATATCTGC,4
TCGA-4P-AA8J,WAPAL,0,.,GRCh37,10,88260246,88260246,+,Missense_Mutation,SNP,...,tolerated_low_confidence(0.77),benign(0),"hmmpanther:PTHR22100:SF13,hmmpanther:PTHR22100",MODERATE,SNV,PASS,NONE,MUTECT|MUSE|VARSCANS,TAAAATACAGT,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WA-A7H4,TSNARE1,0,.,GRCh37,8,143381872,143381872,+,Missense_Mutation,SNP,...,tolerated(0.07),possibly_damaging(0.847),"PROSITE_profiles:PS50192,hmmpanther:PTHR19957:...",MODERATE,SNV,PASS,NONE,MUTECT|MUSE|VARSCANS,CCTCCCGCAGC,3
TCGA-WA-A7H4,TOX,0,.,GRCh37,8,59728263,59728263,+,Silent,SNP,...,.,.,"hmmpanther:PTHR13711,hmmpanther:PTHR13711:SF15...",LOW,SNV,PASS,NONE,RADIA|MUTECT|MUSE|VARSCANS,TTCACGTCAAC,4
TCGA-WA-A7H4,RDH10,0,.,GRCh37,8,74235183,74235183,+,Missense_Mutation,SNP,...,tolerated(0.2),possibly_damaging(0.504),"hmmpanther:PTHR24316:SF309,hmmpanther:PTHR2431...",MODERATE,SNV,PASS,NONE,MUTECT|MUSE,GTATCGGTTCC,2


In [174]:
clean_mut = mut[mut['Variant_Type'] == 'SNP']
Sigtool = clean_mut[['Hugo_Symbol', 'Start_Position', 'Reference_Allele', 'Tumor_Seq_Allele2']]
Sigtool.to_excel('Sigtool.xlsx')
clean_mut = clean_mut[['Chromosome', 'Start_Position', 'Reference_Allele', 'Tumor_Seq_Allele2', 'CONTEXT']]


In [175]:

def get_middle(sequence):
    middle_index = len(sequence) // 2
    if len(sequence) % 2 == 0:
        return sequence[middle_index - 1: middle_index + 1]
    else:
        return sequence[middle_index]
# Apply the function to the 'context' column
clean_mut['middle'] = clean_mut['CONTEXT'].apply(get_middle)


In [176]:
def reverse_complement(sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in sequence[::-1])
# Apply the reverse complement function based on the condition
clean_mut['Complement'] = clean_mut['CONTEXT']
clean_mut.loc[clean_mut['middle'].isin(['G', 'A']), 'Complement'] = clean_mut.loc[clean_mut['middle'].isin(['G', 'A']), 'Complement'].apply(reverse_complement)

In [177]:
def find_middle_three(s):
    length = len(s)
    if length >= 3:
        return s[(length//2)-1:(length//2)+2]
    else:
        return s
clean_mut['Tri'] = clean_mut['Complement'].apply(find_middle_three)

In [178]:
clean_mut

,Chromosome,Start_Position,Reference_Allele,Tumor_Seq_Allele2,CONTEXT,middle,Complement,Tri
patient_id,,,,,,,,
TCGA-4P-AA8J,10,22830863,G,A,CTCTCGCCCTC,G,GAGGGCGAGAG,GCG
TCGA-4P-AA8J,10,33495214,C,T,CATCTCTTCTT,C,CATCTCTTCTT,TCT
TCGA-4P-AA8J,10,6257188,C,T,GTGTTCAACGT,C,GTGTTCAACGT,TCA
TCGA-4P-AA8J,10,75849918,A,T,GAAATATCTGC,A,GCAGATATTTC,ATA
TCGA-4P-AA8J,10,88260246,T,C,TAAAATACAGT,T,TAAAATACAGT,ATA
...,...,...,...,...,...,...,...,...
TCGA-WA-A7H4,8,143381872,C,T,CCTCCCGCAGC,C,CCTCCCGCAGC,CCG
TCGA-WA-A7H4,8,59728263,G,A,TTCACGTCAAC,G,GTTGACGTGAA,ACG
TCGA-WA-A7H4,8,74235183,G,A,GTATCGGTTCC,G,GGAACCGATAC,CCG


In [179]:
clean_mut['Mut'] = (clean_mut['Tri'].str[0] +  "(" + clean_mut['Tri'].str[1] + ">" 
                     + clean_mut['Tumor_Seq_Allele2'].astype(str) + ')' + clean_mut['Tri'].str[2])


In [180]:
clean_mut =  clean_mut[~clean_mut['Mut'].str.contains('(C>C)')]
clean_mut = clean_mut[~clean_mut['Mut'].str.contains('(T>T)')]

/var/folders/x1/0pbqqytx7775hcm5ys9z97y00000gn/T/ipykernel_1132/3941398218.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  clean_mut =  clean_mut[~clean_mut['Mut'].str.contains('(C>C)')]
/var/folders/x1/0pbqqytx7775hcm5ys9z97y00000gn/T/ipykernel_1132/3941398218.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  clean_mut = clean_mut[~clean_mut['Mut'].str.contains('(T>T)')]


In [181]:
clean_mut

,Chromosome,Start_Position,Reference_Allele,Tumor_Seq_Allele2,CONTEXT,middle,Complement,Tri,Mut
patient_id,,,,,,,,,
TCGA-4P-AA8J,10,22830863,G,A,CTCTCGCCCTC,G,GAGGGCGAGAG,GCG,G(C>A)G
TCGA-4P-AA8J,10,33495214,C,T,CATCTCTTCTT,C,CATCTCTTCTT,TCT,T(C>T)T
TCGA-4P-AA8J,10,6257188,C,T,GTGTTCAACGT,C,GTGTTCAACGT,TCA,T(C>T)A
TCGA-4P-AA8J,10,88260246,T,C,TAAAATACAGT,T,TAAAATACAGT,ATA,A(T>C)A
TCGA-4P-AA8J,11,1080567,T,C,TTCGATGGGAA,T,TTCGATGGGAA,ATG,A(T>C)G
...,...,...,...,...,...,...,...,...,...
TCGA-WA-A7H4,8,143381872,C,T,CCTCCCGCAGC,C,CCTCCCGCAGC,CCG,C(C>T)G
TCGA-WA-A7H4,8,59728263,G,A,TTCACGTCAAC,G,GTTGACGTGAA,ACG,A(C>A)G
TCGA-WA-A7H4,8,74235183,G,A,GTATCGGTTCC,G,GGAACCGATAC,CCG,C(C>A)G


In [182]:
np.unique(clean_mut['Mut'])

array(['A(C>A)A', 'A(C>A)C', 'A(C>A)G', 'A(C>A)T', 'A(C>G)A', 'A(C>G)C',
       'A(C>G)G', 'A(C>G)T', 'A(C>T)A', 'A(C>T)C', 'A(C>T)G', 'A(C>T)T',
       'A(T>A)A', 'A(T>A)C', 'A(T>A)G', 'A(T>A)T', 'A(T>C)A', 'A(T>C)C',
       'A(T>C)G', 'A(T>C)T', 'A(T>G)A', 'A(T>G)C', 'A(T>G)G', 'A(T>G)T',
       'C(C>A)A', 'C(C>A)C', 'C(C>A)G', 'C(C>A)T', 'C(C>G)A', 'C(C>G)C',
       'C(C>G)G', 'C(C>G)T', 'C(C>T)A', 'C(C>T)C', 'C(C>T)G', 'C(C>T)T',
       'C(T>A)A', 'C(T>A)C', 'C(T>A)G', 'C(T>A)T', 'C(T>C)A', 'C(T>C)C',
       'C(T>C)G', 'C(T>C)T', 'C(T>G)A', 'C(T>G)C', 'C(T>G)G', 'C(T>G)T',
       'G(C>A)A', 'G(C>A)C', 'G(C>A)G', 'G(C>A)T', 'G(C>G)A', 'G(C>G)C',
       'G(C>G)G', 'G(C>G)T', 'G(C>T)A', 'G(C>T)C', 'G(C>T)G', 'G(C>T)T',
       'G(T>A)A', 'G(T>A)C', 'G(T>A)G', 'G(T>A)T', 'G(T>C)A', 'G(T>C)C',
       'G(T>C)G', 'G(T>C)T', 'G(T>G)A', 'G(T>G)C', 'G(T>G)G', 'G(T>G)T',
       'T(C>A)A', 'T(C>A)C', 'T(C>A)G', 'T(C>A)T', 'T(C>G)A', 'T(C>G)C',
       'T(C>G)G', 'T(C>G)T', 'T(C>T)A', 'T(C>T)C', 

In [183]:
mut_matrix = clean_mut[['Mut']]


In [184]:
mut_encoded_matrix = pd.pivot_table(mut_matrix, index='patient_id', columns='Mut', aggfunc='size', fill_value=0)

In [185]:
mut_encoded_matrix

Mut,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,...,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,7,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,0
TCGA-BA-4074,5,1,5,1,0,1,0,1,2,1,...,0,0,0,0,1,0,0,0,0,0
TCGA-BA-4075,3,5,3,3,0,0,0,1,1,5,...,0,1,0,0,0,1,2,0,0,2
TCGA-BA-4076,9,5,7,5,0,1,1,2,7,8,...,1,1,1,1,3,0,2,2,4,1
TCGA-BA-4077,1,1,4,0,0,0,0,0,1,1,...,0,1,0,1,0,1,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JT,4,2,2,1,1,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-UF-A7JV,0,0,1,1,0,0,0,0,2,1,...,1,1,0,1,0,0,0,0,0,0
TCGA-UP-A6WW,0,0,3,1,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,1


In [186]:
model = NMF(n_components=2, init='random', random_state=0) #k = 2
W = model.fit_transform(mut_encoded_matrix)
coefficient = model.components_

In [187]:
sum(W[0])

1.1005117613863566

In [188]:
coefficient

array([[ 0.56722223,  1.31511637,  0.60139873,  0.58764561,  0.09896426,
         0.08758692,  0.05889868,  0.05366756,  0.78292256,  1.21054017,
         0.55243039,  0.65112137,  0.10640799,  0.08735992,  0.06905053,
         0.13779847,  0.24397149,  0.10045795,  0.23844626,  0.16279404,
         0.18453445,  0.09052446,  0.30984618,  0.13254112,  6.99903108,
         6.1584145 ,  4.20593906,  5.28006595,  0.07078921,  0.11282162,
         0.11838886,  0.15760536,  6.56047703,  5.8314137 ,  3.63445891,
         4.5174503 ,  0.08964371,  0.11472128,  0.28621725,  0.19605008,
         0.12135508,  0.14662941,  0.42460958,  0.26825638,  0.15134439,
         0.18858108,  0.27323833,  0.21741112,  0.57284338,  1.32325507,
         0.72192989,  0.80202075,  0.05662676,  0.0861485 ,  0.10759554,
         0.03968284,  0.57206066,  1.3871787 ,  0.70959838,  0.62610611,
         0.08105775,  0.0547981 ,  0.19047013,  0.06332546,  0.23513664,
         0.14094051,  0.27735654,  0.33352467,  0.2

In [87]:
column_sums = np.sum(coefficient, axis=0)

# Index of the original feature most influenced by the hidden features
most_influenced_feature = np.argmax(column_sums)
most_influenced_feature

72

In [88]:
mut_matrix.iloc[72]

Mut    G(T>C)C
Name: (TCGA-4P-AA8J, KDM1A), dtype: object

In [100]:
# 100 possion resample
n_samples = 1
indices = np.random.poisson(lam=mut_encoded_matrix.shape[0], size=n_samples)

# Sample the DataFrame using the Poisson indices
df_resampled = mut_encoded_matrix.iloc[indices]


IndexError: positional indexers are out-of-bounds

In [ ]:
df_resampled

In [97]:
from SigProfilerExtractor import sigpro as sig
path_to_example_table = sig.importdata("matrix")
data = path_to_example_table 

ModuleNotFoundError: No module named 'SigProfilerExtractor'